In [1]:
import sqlite3
import pandas as pd
from sqlite3 import Error

In [2]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print("Got DB Connection")
    except Error as e:
        print(e)
 
    return conn

In [3]:
def get_db_results(conn, query):
    """ Return db results based on query
    :param conn: the Connection object
    :param query: Query to execute
    :return: None
    """
    df = pd.read_sql_query(query, conn)
    print(df)

In [4]:
database = "sqlite_db_pythonsqlite.db"
 
# create a database connection
conn = create_connection(database)

Got DB Connection


In [5]:
# Q10: Produce a list of facilities with a total revenue less than 1000.

query = """
    select bkgfac.name AS facility_name, SUM(bkgfac.faccost) total_revenue
        from Members AS mem
        INNER JOIN (
        select bkg.memid, fac.facid, fac.name, 
            case when bkg.memid = 0 then fac.guestcost else fac.membercost end faccost
        from Bookings as bkg
        inner join Facilities as fac on fac.facid = bkg.facid
        ) AS bkgfac ON bkgfac.memid = mem.memid
        GROUP BY bkgfac.facid, bkgfac.name 
        HAVING total_revenue < 1000
        ORDER BY total_revenue
    """

with conn: 
    print("10. Produce a list of facilities with a total revenue less than 1000")
    get_db_results(conn, query)

10. Produce a list of facilities with a total revenue less than 1000
     facility_name  total_revenue
0     Table Tennis           90.0
1    Snooker Table          115.0
2       Pool Table          265.0
3  Badminton Court          604.5


In [6]:
# Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

query = """
        select CONCAT(mem.firstname, ' ', mem.surname) member_name, CONCAT(rec.firstname, ' ', rec.surname) recommended_by 
        from Members mem
        INNER JOIN Members rec ON rec.memid = mem.recommendedby
        where mem.recommendedby is not null and mem.recommendedby != ''
        order by mem.surname, mem.firstname    
    """

with conn: 
    print("11. Produce a report of members and who recommended them in alphabetic surname,firstname order")
    get_db_results(conn, query)

11. Produce a report of members and who recommended them in alphabetic surname,firstname order
                member_name     recommended_by
0            Florence Bader    Ponder Stibbons
1                Anne Baker    Ponder Stibbons
2             Timothy Baker     Jemima Farrell
3                Tim Boothe         Tim Rownam
4            Gerald Butters       Darren Smith
5               Joan Coplin      Timothy Baker
6             Erica Crumpet        Tracy Smith
7                Nancy Dare    Janice Joplette
8           Matthew Genting     Gerald Butters
9                 John Hunt  Millicent Purview
10              David Jones    Janice Joplette
11            Douglas Jones        David Jones
12          Janice Joplette       Darren Smith
13           Anna Mackenzie       Darren Smith
14             Charles Owen       Darren Smith
15             David Pinker     Jemima Farrell
16        Millicent Purview        Tracy Smith
17         Henrietta Rumney    Matthew Genting
18         R

In [7]:
# Q12: Find the facilities with their usage by member, but not guests

query = """
        select fac.name as facitlity_name, count(*) as usage_count from Bookings bkg
        inner join Facilities fac ON fac.facid = bkg.facid
        where bkg.memid != 0
        group by fac.name 
        order by usage_count desc   
    """

with conn: 
    print("12: Find the facilities with their usage by member, but not guests")
    get_db_results(conn, query)

12: Find the facilities with their usage by member, but not guests
    facitlity_name  usage_count
0       Pool Table          783
1    Snooker Table          421
2   Massage Room 1          421
3     Table Tennis          385
4  Badminton Court          344
5   Tennis Court 1          308
6   Tennis Court 2          276
7     Squash Court          195
8   Massage Room 2           27


In [8]:
# Q13: Find the facilities usage by month, but not guests

query = """
        select fac.name as facitlity_name, strftime("%m", bkg.starttime) month, count(*) usage_count from Bookings bkg
        inner join Facilities fac ON fac.facid = bkg.facid
        where bkg.memid != 0
        group by fac.name,  strftime("%m", bkg.starttime)
        order by facitlity_name, bkg.starttime  
    """

with conn: 
    print("13: Find the facilities usage by month, but not guests")
    get_db_results(conn, query)

13: Find the facilities usage by month, but not guests
     facitlity_name month  usage_count
0   Badminton Court    07           51
1   Badminton Court    08          132
2   Badminton Court    09          161
3    Massage Room 1    07           77
4    Massage Room 1    08          153
5    Massage Room 1    09          191
6    Massage Room 2    07            4
7    Massage Room 2    08            9
8    Massage Room 2    09           14
9        Pool Table    07          103
10       Pool Table    08          272
11       Pool Table    09          408
12    Snooker Table    07           68
13    Snooker Table    08          154
14    Snooker Table    09          199
15     Squash Court    07           23
16     Squash Court    08           85
17     Squash Court    09           87
18     Table Tennis    07           48
19     Table Tennis    08          143
20     Table Tennis    09          194
21   Tennis Court 1    07           65
22   Tennis Court 1    08          111
23   Tenn